# Вебинар 6. Двухуровневые модели рекомендаций


Код для src, utils, metrics вы можете скачать из [этого](https://github.com/geangohn/recsys-tutorial) github репозитория

In [1]:
!pip install implicit

In [2]:
!pip install graphviz==0.19

In [34]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline


from numpy import bincount, log, log1p, sqrt

# Для работы с матрицами
from scipy.sparse import coo_matrix, csr_matrix

# Матричная факторизация
from implicit import als

from implicit.als import AlternatingLeastSquares
from implicit.nearest_neighbours import ItemItemRecommender  # нужен для одного трюка
from implicit.nearest_neighbours import bm25_weight, tfidf_weight

# Модель второго уровня
from lightgbm import LGBMClassifier

import os, sys
sys.path.append('C:/Users/sklod/OneDrive/Рабочий стол/STUDY/!Теория/Q4.Рекомендательные системы/src')

# module_path = os.path.abspath(os.path.join(os.pardir))
# if module_path not in sys.path:
#     sys.path.append(module_path)

# Написанные нами функции

from src.metrics import precision_at_k, recall_at_k
from src.utils import prefilter_items
from src.recommenders import MainRecommender

In [39]:
data = pd.read_csv('C:/Users/sklod/OneDrive/Рабочий стол/STUDY/!Теория/Q4.Рекомендательные системы/retail_train.csv')
item_features = pd.read_csv('C:/Users/sklod/OneDrive/Рабочий стол/STUDY/!Теория/Q4.Рекомендательные системы/product.csv')
user_features = pd.read_csv('C:/Users/sklod/OneDrive/Рабочий стол/STUDY/!Теория/Q4.Рекомендательные системы/hh_demographic.csv')

# column processing
item_features.columns = [col.lower() for col in item_features.columns]
user_features.columns = [col.lower() for col in user_features.columns]

item_features.rename(columns={'product_id': 'item_id'}, inplace=True)
user_features.rename(columns={'household_key': 'user_id'}, inplace=True)


# Важна схема обучения и валидации!
# -- давние покупки -- | -- 6 недель -- | -- 3 недель -- 
# подобрать размер 2-ого датасета (6 недель) --> learning curve (зависимость метрики recall@k от размера датасета)

# модель 1 уровня - обучается на давних покупках, валидируется на промежуточных 6 неделях
# модель 2 уровня - обучается на промежуточных 6 неделях, валидируется на последних 3 неделях

val_lvl_1_size_weeks = 6
val_lvl_2_size_weeks = 3

# берем данные для тренировки модели 1 уровня
data_train_lvl_1 = data[data['week_no'] < data['week_no'].max() - (val_lvl_1_size_weeks + val_lvl_2_size_weeks)]

# берем данные для валидации matching модели
data_val_lvl_1 = data[(data['week_no'] >= data['week_no'].max() - (val_lvl_1_size_weeks + val_lvl_2_size_weeks)) &
                      (data['week_no'] < data['week_no'].max() - (val_lvl_2_size_weeks))]

# берем данные для тренировки модели 2 уровня
data_train_lvl_2 = data_val_lvl_1.copy()  # Для наглядности. Далее мы добавим изменения, и они будут отличаться

# берем данные для теста ranking, matching модели
data_val_lvl_2 = data[data['week_no'] >= data['week_no'].max() - val_lvl_2_size_weeks]

data_train_lvl_1.head(2)

,user_id,basket_id,day,item_id,quantity,sales_value,store_id,retail_disc,trans_time,week_no,coupon_disc,coupon_match_disc
0,2375,26984851472,1,1004906,1,1.39,364,-0.6,1631,1,0.0,0.0
1,2375,26984851472,1,1033142,1,0.82,364,0.0,1631,1,0.0,0.0


In [40]:
def print_stats_data(df_data, name_df):
    print(name_df)
    print(f"Shape: {df_data.shape} Users: {df_data['user_id'].nunique()} Items: {df_data['item_id'].nunique()}")

In [41]:
# видим разброс по пользователям и товарам
print_stats_data(data_train_lvl_1,'data_train_lvl_1')
print_stats_data(data_val_lvl_1,'data_val_lvl_1')
print_stats_data(data_train_lvl_2,'data_train_lvl_2')
print_stats_data(data_val_lvl_2,'data_val_lvl_2')

data_train_lvl_1
Shape: (2108779, 12) Users: 2498 Items: 83685
data_val_lvl_1
Shape: (169711, 12) Users: 2154 Items: 27649
data_train_lvl_2
Shape: (169711, 12) Users: 2154 Items: 27649
data_val_lvl_2
Shape: (118314, 12) Users: 2042 Items: 24329


In [42]:
# Префильтрация items

n_items_before = data_train_lvl_1['item_id'].nunique()

data_train_lvl_1 = prefilter_items(data_train_lvl_1, item_features=item_features, take_n_popular=5000)

n_items_after = data_train_lvl_1['item_id'].nunique()
print('Decreased # items from {} to {}'.format(n_items_before, n_items_after))

Decreased # items from 83685 to 5001


In [8]:
#d#ata_train_lvl_1.user_id.values

In [23]:
# # ищем общих пользователей
# common_users = list(set(data_train_lvl_1.user_id.values)&(set(data_val_lvl_1.user_id.values))\
#                     &set(data_val_lvl_2.user_id.values))

# data_train_lvl_1 = data_train_lvl_1[data_train_lvl_1.user_id.isin(common_users)]
# data_val_lvl_1 = data_val_lvl_1[data_val_lvl_1.user_id.isin(common_users)]
# data_train_lvl_2 = data_train_lvl_2[data_train_lvl_2.user_id.isin(common_users)]
# data_val_lvl_2 = data_val_lvl_2[data_val_lvl_2.user_id.isin(common_users)]

# print_stats_data(data_train_lvl_1,'data_train_lvl_1')
# print_stats_data(data_val_lvl_1,'data_val_lvl_1')
# print_stats_data(data_train_lvl_2,'data_train_lvl_2')
# print_stats_data(data_val_lvl_2,'data_val_lvl_2')

data_train_lvl_1
Shape: (784420, 13) Users: 1915 Items: 4999
data_val_lvl_1
Shape: (163261, 12) Users: 1915 Items: 27118
data_train_lvl_2
Shape: (163261, 12) Users: 1915 Items: 27118
data_val_lvl_2
Shape: (115989, 12) Users: 1915 Items: 24042


In [43]:
# Инициализируем класс Main Recommender
# В нем предусмотрено сразу обучение на обучающей выборке модели 1 уровня (ALS модель)

recommender = MainRecommender(data_train_lvl_1)

In [44]:
recommender.get_als_recommendations(2375, N=5)

[899624, 1044078, 1106523, 871756, 5569230]

In [45]:
recommender.get_own_recommendations(2375, N=5)

[948640, 918046, 847962, 907099, 873980]

In [46]:
recommender.get_similar_items_recommendation(2375, N=5)

[1046545, 917816, 1042907, 842125, 15778319]

In [47]:
recommender.get_similar_users_recommendation(2375, N=5)

[974265, 12523928, 820612, 918638, 897671]

### Задание 1

A) Попробуйте различные варианты генерации кандидатов. Какие из них дают наибольший recall@k ?
- Пока пробуем отобрать 50 кандидатов (k=50)
- Качество измеряем на data_val_lvl_1: следующие 6 недель после трейна

Дают ли own recommendtions + top-popular лучший recall?  

B)* Как зависит recall@k от k? Постройте для одной схемы генерации кандидатов эту зависимость для k = {20, 50, 100, 200, 500}  
C)* Исходя из прошлого вопроса, как вы думаете, какое значение k является наиболее разумным?


In [48]:
# Создаем результиующий датафрейм для расчета результата метрики на тестовых данных
# Для 1 уровня тестовый набор данных - промежуточные 6 недель - data_val_lvl1

result_eval_lvl_1 = data_val_lvl_1.groupby('user_id')['item_id'].unique().reset_index()
result_eval_lvl_1.columns=['user_id', 'actual']
result_eval_lvl_1.head(2)

,user_id,actual
0,1,"[853529, 865456, 867607, 872137, 874905, 87524..."
1,2,"[15830248, 838136, 839656, 861272, 866211, 870..."


In [49]:
# Функция для получение рекомендаций
def make_recommendations(df_result, recommend_model, N=50, user_col='user_id'):
    return df_result[user_col].apply(lambda x: recommend_model(x, N=N))

In [50]:
# Функция для расчета recall@k

def calc_recall_at_k(df_data, top_k):
    for col_name in df_data.columns[2:]:
        yield col_name, round((df_data.apply(lambda row: recall_at_k(row[col_name], row['actual'], k=top_k), axis=1).mean()), 4)

# Фнукция для расчета precision@k
def calc_precision_at_k(df_data, top_k):
    for col_name in df_data.columns[2:]:
        yield col_name, df_data.apply(lambda row: precision_at_k(row[col_name], row['actual'], k=top_k), axis=1).mean()

In [51]:
%%time

models = {'als_rec': recommender.get_als_recommendations,
          'own_rec': recommender.get_own_recommendations, 
          'similar_item_rec': recommender.get_similar_items_recommendation, 
          'similar_user_rec': recommender.get_similar_users_recommendation}

for col_name, model in models.items():
    result_eval_lvl_1[col_name] = make_recommendations(result_eval_lvl_1, model)

IndexError: index 2496 is out of bounds for axis 0 with size 2496

In [33]:
result_eval_lvl_1.head(10)

,user_id,actual,als_rec,own_rec,similar_item_rec
0,1,"[853529, 865456, 867607, 872137, 874905, 87524...","[6534030, 885290, 1062572, 1097909, 5569374, 8...","[856942, 9297615, 5577022, 877391, 9655212, 10...","[839818, 1007512, 9297474, 5577022, 9803545, 9..."
1,6,"[1024306, 1102949, 6548453, 835394, 940804, 96...","[878996, 933637, 854852, 1026118, 863632, 9652...","[13003092, 995598, 923600, 972416, 1084036, 11...","[948650, 5569845, 8357613, 941361, 1074754, 11..."
2,7,"[836281, 843306, 845294, 914190, 920456, 93886...","[10285022, 1063739, 1041688, 9338009, 6443332,...","[998519, 894360, 7147142, 9338009, 896666, 939...","[957411, 949023, 1044078, 880427, 12352330, 83..."
3,8,"[868075, 886787, 945611, 1005186, 1008787, 101...","[916122, 1139782, 938596, 981660, 844179, 9982...","[12808385, 939860, 981660, 7410201, 5577022, 6...","[5569845, 8090532, 1044078, 12301073, 1130581,..."
4,9,"[883616, 1029743, 1039126, 1051323, 1082772, 1...","[970866, 861899, 901067, 6534030, 1042942, 111...","[872146, 918046, 9655676, 985622, 1056005, 109...","[6424471, 1074754, 857503, 982537, 983096, 713..."
5,13,"[6544236, 822407, 908317, 1056775, 1066289, 11...","[8090513, 13158992, 1048068, 5569471, 13159268...","[965772, 9488065, 10342382, 6554400, 862070, 1...","[1074754, 998352, 838882, 6553237, 1132911, 80..."
6,14,"[917277, 981760, 878234, 925514, 986394, 10220...","[910673, 1025611, 1131344, 1127758, 846823, 11...","[902377, 822161, 874563, 1123106, 8090610, 138...","[1074754, 910673, 897954, 1025611, 990335, 863..."
7,15,"[996016, 1014509, 1044404, 1087353, 976199, 10...","[1082185, 933637, 1042616, 1034956, 12263857, ...","[823576, 1052975, 1053530, 1071196, 1010051, 1...","[857503, 1074754, 1135476, 1055297, 866211, 10..."
8,16,"[860361, 12263439, 13007710, 866227, 1084551, ...","[1100533, 923149, 1017325, 904493, 954495, 101...","[820486, 1138596, 5707857, 1029743, 1106523, 5...","[904493, 1017325, 1029743, 1106523, 5569230, 9..."
9,17,"[826784, 833351, 845208, 850529, 916050, 92355...","[12810391, 899624, 896613, 5568072, 6534030, 1...","[924004, 5567874, 1055403, 5568072, 939860, 11...","[5567874, 5568072, 1117602, 7409941, 957951, 1..."


In [34]:
# Переходим к расчету recall@k - Recall@50 (N=50 по умолчанию в make_recommendations)

top_k_recall = 50

sorted(calc_recall_at_k(result_eval_lvl_1, top_k_recall), key=lambda x: x[1],reverse=True)

[('own_rec', 0.0617), ('als_rec', 0.047), ('similar_item_rec', 0.0312)]

Вывод: Наибольший recall при k=50 кандидатов в рекомендацию дает модель own_recommendations

In [35]:
# Изучим, как будет меняться метрика в зависимости от количества кандидатов

top_k_list = [20, 50, 100, 200, 500]

for k in top_k_list:
    result_eval_lvl_1 = data_val_lvl_1.groupby('user_id')['item_id'].unique().reset_index()
    result_eval_lvl_1.columns=['user_id', 'actual']
    
    for column_name, model in models.items():
        result_eval_lvl_1[column_name] = make_recommendations(result_eval_lvl_1, model, N=k)
        
    print(f'{k} кандидатов: \n{sorted(calc_recall_at_k(result_eval_lvl_1, k), key=lambda x: x[1],reverse=True)}')

ValueError: userid is out of bounds of the user_items matrix

Вывод:

очевидно, что с увеличением количества кандидатов метрика пропорционально растет. Градация по убываю метрики среди моделей сохраняется (лучший результат показывает модель own_recommenders + top-popular (Если кол-во рекоммендаций < N, то дополняем их топ-популярными), самый слабый результат - модель similar_user_rec).

Однако, чем больше кандидатов, тем сложнее модель и тем дольше она обучается, что также отражается на модели следующего уровня. Остановимся пока на количестве кандидатов - 50

### Задание 2.

Обучите модель 2-ого уровня, при этом:
    - Добавьте минимум по 2 фичи для юзера, товара и пары юзер-товар
    - Измерьте отдельно precision@5 модели 1-ого уровня и двухуровневой модели на data_val_lvl_2
    - Вырос ли precision@5 при использовании двухуровневой модели?

In [ ]:
# youe code

# Задание 3


X_val подготовить из data_val_lvl_2 также как и X_train из targets_lvl_2
val_preds = lgb.predict(X_val) #, где X_val - датасет подготовленный по аналогии с data_train_lvl_2

Далее оценить на валидационном множестве precision

Сравнить с трейном (у меня было 0.78782)

In [ ]:
# your_code

In [ ]:
def precision_at_k(recommended_list, bought_list, k=5):
    
    bought_list = np.array(bought_list)
    recommended_list = np.array(recommended_list)
    
    bought_list = bought_list  # Тут нет [:k] !!
    try:
        recommended_list = recommended_list[:k]
    except:
        recommended_list = []
    
    flags = np.isin(bought_list, recommended_list)
    
    precision = flags.sum() / len(recommended_list)
    
    
    return precision

In [ ]:
train_data_result = X_train[X_train['y_true']>0].groupby('user_id')['item_id'].agg(list).reset_index()
preds = X_train[X_train['preds']>0].groupby('user_id')['item_id'].agg(list).reset_index()
train_data_result.rename(columns={'item_id':'actual'}, inplace = True)
train_data_result = train_data_result.merge(preds, how='left', on='user_id')
train_data_result.rename(columns={'item_id':'lightGBM'}, inplace = True)

In [ ]:
train_data_result.apply(lambda row: precision_at_k(row['lightGBM'], row['actual']), axis=1).mean()

### Финальный проект

Мы уже прошли всю необходимуб теорию для финального проекта. Проект осуществляется на данных из вебинара (данные считаны в начале ДЗ).
Рекомендуем вам **начать делать проект сразу после этого домашнего задания**
- Целевая метрика - precision@5. Порог для уcпешной сдачи проекта precision@5 > 25%
- Будет public тестовый датасет, на котором вы сможете измерять метрику
- Также будет private тестовый датасет для измерения финального качества
- НЕ обязательно, но крайне желательно использовать 2-ух уровневые рекоммендательные системы в проекте
- Вы сдаете код проекта в виде github репозитория и csv файл с рекомендациями 